In [ ]:
import torchaudio as ta
from chatterbox_infer.tts import ChatterboxTTS
import time

model = ChatterboxTTS.from_pretrained(device="cuda")

In [ ]:
from IPython.display import Audio

text = "hm.."
out = 'hmhm.wav'
wav, tokens = model.generate(text, audio_prompt_path='/workspace/chatterbox/sesame.wav')
print(wav.shape[-1]/24000)
ta.save(out, wav, model.sr)
display(Audio(out))

In [ ]:
from IPython.display import Audio
import torch

display(Audio(wav, rate=24000))
bi=0
all_audios = []
for i in [20, 40, 60, 80, 100, 120, 140, 160]:
    cwav, _ = model.s3gen.inference(
        speech_tokens=tokens[bi:i],
        ref_dict=model.conds.gen,
    )
    all_audios.append(cwav.detach().cpu())
    # display(Audio(cwav.detach().cpu(), rate=24000))
    bi = i

In [ ]:
summed_wav = torch.concat(all_audios, dim=1)
display(Audio(summed_wav, rate=24000))

In [ ]:
wav.shape

In [ ]:
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
import time
from IPython.display import Audio

model = ChatterboxTTS.from_pretrained(device="cuda")

In [ ]:
text = "Oh, I really love it.. y'know, it's amazing."

last_length = 0
st = time.time()
wavs = []
async for evt in model.generate_stream(text, audio_prompt_path='test-1.wav'):
    if evt["type"] == "chunk":
        wav = evt["audio"]
        wavs.append(wav[:, max(0, last_length-1000):])
        print(f"[{(wav.shape[-1]-last_length)/24000}s] - {time.time() - st}")
        last_length = wav.shape[-1]
        st = time.time()
    elif evt["type"] == "eos":
        print("✅ 스트리밍 끝!")

In [ ]:
import torch, time

SR = 24000
OVERLAP = int(0.05 * SR)  # 50ms

text = "Oh, I really love it.. y'know, it's amazing."

last_length = 0          # 모델이 지금까지 만든 전체 wav 길이
last_tail = None         # 직전 출력 청크의 꼬리(OVERLAP 샘플)
st = time.time()

wavs = []                # overlap-add로 섞은 최종 출력 청크들

async for evt in tts_model.generate_stream(text, audio_prompt_path='test-1.wav'):
    if evt["type"] == "chunk":
        wav = evt["audio"]                    # shape: (ch, T_total_so_far)
        device = wav.device
        dtype = wav.dtype

        # 이번에 "새로 추가된" 구간만 잘라오기
        new_total = wav.shape[-1]
        delta = new_total - last_length
        if delta <= 0:
            continue  # 새로 생긴 게 없으면 스킵

        new_part = wav[:, last_length:new_total]  # (ch, delta)

        if last_tail is None:
            # 첫 청크면 바로 내보냄
            out_chunk = new_part
        else:
            # 겹치는 길이 L (= min(OVERLAP, 새로 생긴 길이, last_tail 길이))
            L = min(OVERLAP, new_part.shape[-1], last_tail.shape[-1])
            if L > 0:
                # last_tail의 마지막 L 샘플 ↔ new_part의 앞 L 샘플을 교차페이드
                fade_in  = torch.linspace(0, 1, L, device=device, dtype=dtype)
                fade_out = 1.0 - fade_in

                mixed = last_tail[:, -L:] * fade_out + new_part[:, :L] * fade_in
                tail  = new_part[:, L:]  # 비겹침 뒷부분

                out_chunk = torch.cat([mixed, tail], dim=-1)
            else:
                # 겹칠 게 없으면 그냥 이어붙임
                out_chunk = new_part

        # 다음 교차페이드를 위해 꼬리 갱신
        # (모델 기준 전체 wav의 최신 꼬리를 쓰는 게 안전)
        new_tail_start = max(0, new_total - OVERLAP)
        last_tail = wav[:, new_tail_start:new_total].detach()

        # 사용자 출력/저장을 위해 overlap-add 결과만 모음
        wavs.append(out_chunk)

        print(f"[{out_chunk.shape[-1]/SR:.3f}s] - {time.time() - st:.3f}")
        last_length = new_total
        st = time.time()

    elif evt["type"] == "eos":
        print("✅ 스트리밍 끝!")

In [ ]:
import torch

display(Audio(wav.cpu().numpy(), rate=24000))   # 예: 오디오 출력 함수
display(Audio(torch.concat(wavs, dim=-1).cpu().numpy(), rate=24000))   # 예: 오디오 출력 함수

In [ ]:
st = time.time()
model.t3.speech_pos_emb.get_fixed_embedding(213)
time.time() - st

import torch

st = time.time()
emb = torch.nn.Embedding(113, 512)
emb(torch.tensor(12))
time.time() - st

In [ ]:
st = time.time()
torch.tensor(132).to('cuda')
time.time() - st

In [ ]:
logits = torch.randn((132001))
st = time.time()
probs = torch.softmax(logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)
print(time.time() - st)

st = time.time()
dist = torch.distributions.Categorical(logits=logits)  # GPU에서 동작
next_token = dist.sample()  # (B, 1) 형태 맞추기
print(time.time() - st)


In [ ]:
import librosa
import time

audio, sr = librosa.load('test-1.wav')

text = "Oh"
st = time.time()
wav, tokens = model.generate(text, audio_prompt_path=audio)
print(wav.shape, tokens.shape, time.time() - st)
# ta.save("test-1.wav", wav, model.sr)
display(Audio('test-1.wav'))

In [ ]:
import torchaudio as ta
from chatterbox_infer.tts import ChatterboxTTS
import time

model = ChatterboxTTS.from_pretrained(device="cuda")

In [ ]:
from IPython.display import Audio
import librosa
import torch

audio, _ = librosa.load('/workspace/chatterbox/sesame.wav', sr=16000, mono=True)

text = "I want to go home now please let me go."
wav, tokens = model.generate(text, audio_prompt_path=audio)
print(wav.shape[-1]/24000)

out = 'just.wav'
ta.save(out, wav, model.sr)
display(Audio(out))

In [ ]:
display(Audio(audio, rate=16000))

In [15]:
import queue

aa = queue.Queue()

In [16]:
aa.get()

KeyboardInterrupt: 